In [1]:
!pip install pyarrow

In [2]:
import pandas as pd

import seaborn as sns

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

### Downloading the Data

### Q1. Downloading the data

We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

In [3]:
df_train = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_val = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [4]:
df_train.head(2)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.0,0.5,0.0,0.0,1.0,14.3,2.5,0.0
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.0,0.5,4.0,0.0,1.0,16.9,2.5,0.0


In [5]:
df_val.head(2)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.3,1.0,N,142,163,2,4.4,3.5,0.5,0.0,0.0,1.0,9.4,2.5,0.0
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.0,1.0,N,71,71,4,-3.0,-1.0,-0.5,0.0,0.0,-1.0,-5.5,0.0,0.0


In [31]:
len(df_train.columns), len(df_val.columns)

(20, 20)

In [32]:
num_columns_original_train = len(df_train.columns)

print(f'The total number of columns on training data is {num_columns_original_train}')

The total number of columns on training data is 20


### Q2. Computing duration

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [7]:
df_train['duration'] = df_train['tpep_dropoff_datetime'] - df_train['tpep_pickup_datetime']
df_train['duration'] = df_train.duration.apply(lambda td: td.total_seconds() / 60)

In [8]:
df_train.head(2)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.0,0.5,0.0,0.0,1.0,14.3,2.5,0.0,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.0,0.5,4.0,0.0,1.0,16.9,2.5,0.0,6.316667


In [9]:
std_dev_duration = df_train['duration'].std()

print(f'The Standard Deviation of rides in minutes, in January, is: {std_dev_duration}')

The Standard Deviation of rides in minutes, in January, is: 42.594351241920904


### Q3. Dropping outliers

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [10]:
#df_filtered = df[df['duration'].between(pd.Timedelta(minutes=1), pd.Timedelta(minutes=60))]
df_train_filtered = df_train[(df_train.duration >= 1) & (df_train.duration <= 60)]

In [11]:
percentage_kept = (df_train_filtered.shape[0] / df_train.shape[0]) * 100
formated_percentage_kept = "{:.2f}".format(percentage_kept)

print(f"The total percentage of values kept after removing outliers were: {formated_percentage_kept}%")

The total percentage of values kept after removing outliers were: 98.12%


### Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

In [12]:
df_train = df_train_filtered

In [13]:
# Define features
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

In [14]:
# Transform categorical features into strings
df_train[categorical] = df_train[categorical].astype(str)

In [15]:
# Turn dataframe into dictionary
train_dicts = df_train[categorical + numerical].to_dict(orient='records')

In [16]:
dv = DictVectorizer()

X_train = dv.fit_transform(train_dicts)

In [17]:
num_columns = X_train.shape[1]

print(f'The number of columns in the resulting matrix is: {num_columns}')

The number of columns in the resulting matrix is: 516


### Q5. Training a model

Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters
Calculate the RMSE of the model on the training data
What's the RMSE on train?

In [19]:
target = 'duration'
y_train = df_train[target].values

In [20]:
lr = LinearRegression()

lr.fit(X_train, y_train)

LinearRegression()

In [21]:
y_pred = lr.predict(X_train)

In [22]:
# sns.displot(y_pred, label='prediction')
# sns.displot(y_train, label='actual')

training_rmse = mean_squared_error(y_train, y_pred, squared=False)

print(f'RMSE of the model on the training data is: {training_rmse}')

RMSE of the model on the training data is: 7.649131698810958


### Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [23]:
df_val.head(2)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.3,1.0,N,142,163,2,4.4,3.5,0.5,0.0,0.0,1.0,9.4,2.5,0.0
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.0,1.0,N,71,71,4,-3.0,-1.0,-0.5,0.0,0.0,-1.0,-5.5,0.0,0.0


In [24]:
# Correct duration of rides
df_val['duration'] = df_val['tpep_dropoff_datetime'] - df_val['tpep_pickup_datetime']
df_val['duration'] = df_val.duration.apply(lambda td: td.total_seconds() / 60)

df_val_filtered = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]

df_val = df_val_filtered

# Transform categorical features into strings
df_val[categorical] = df_val[categorical].astype(str)

In [25]:
# dv_val = DictVectorizer()

# Turn dataframe into dictionary
val_dicts = df_val[categorical + numerical].to_dict(orient='records')

# Transform validation data
X_val = dv.transform(val_dicts)

In [26]:
# Get y validation
target = 'duration'
y_val = df_val[target].values

In [27]:
lr = LinearRegression()

lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

val_rmse = mean_squared_error(y_val, y_pred, squared=False)

print(f'RMSE of the model on the validation data is: {val_rmse}')

RMSE of the model on the validation data is: 7.8114398467122825


## Final Answers

In [37]:
# Question 01
print("Question 01")
print(f'The total number of columns on training data is {num_columns_original_train}\n')

#Question 02
print("Question 02")
print(f'The Standard Deviation of rides in minutes, in January, is: {std_dev_duration}\n')

#Question 03
print("Question 03")
print(f"The total percentage of values kept after removing outliers were: {formated_percentage_kept}%\n")

#Question 04
print("Question 04")
print(f'The number of columns in the resulting matrix is: {num_columns}\n')

#Question 05
print("Question 05")
print(f'RMSE of the model on the training data is: {training_rmse}\n')

#Question 06
print("Question 06")
print(f'RMSE of the model on the validation data is: {val_rmse}')

Question 01
The total number of columns on training data is 20

Question 02
The Standard Deviation of rides in minutes, in January, is: 42.594351241920904

Question 03
The total percentage of values kept after removing outliers were: 98.12%

Question 04
The number of columns in the resulting matrix is: 516

Question 05
RMSE of the model on the training data is: 7.649131698810958

Question 06
RMSE of the model on the validation data is: 7.8114398467122825
